In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn.metrics

# Input data files are available in the read-only "../input/" directory
# For examasdfple, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook985089b5b5/bert-base-uncasedNetMLE_3.pt
/kaggle/input/notebook985089b5b5/__results__.html
/kaggle/input/notebook985089b5b5/__notebook__.ipynb
/kaggle/input/notebook985089b5b5/__output__.json
/kaggle/input/notebook985089b5b5/bert-base-uncasedNetMLE_2.pt
/kaggle/input/notebook985089b5b5/custom.css
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/*

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip


In [3]:
# !pip install transformers > /dev/null

In [4]:
import pandas as pd, numpy as np
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
# import tensorflow as tf
# tf.summary.FileWriter('log_dir') 
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
# import wandb
# os.environ["WANDB_API_KEY"] = '3ace0586afdcb3d308f9316e385f913a9463bdaa'

In [5]:
train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
# subm = pd.read_csv('../input/sample_submission.csv')

train_split1 = train[(train.toxic!=0)|(train.severe_toxic!=0)|(train.obscene!=0)|(train.threat!=0)|(train.insult!=0)|(train.identity_hate!=0)]
train_split2 = train[(train.toxic==0)&(train.severe_toxic==0)&(train.obscene==0)&(train.threat==0)&(train.insult==0)&(train.identity_hate==0)]
train_split2 = train_split2.sample(1200).reset_index(drop=True)

train_split1['none'] = 0
train_split2['none'] = 1

train = pd.concat([train_split1.sample(5000), train_split2])
print(len(train))

print(len(train_split1), len(train_split2))

train = train.reset_index(drop=True)
train.sample(20)

6200
16225 1200


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
5686,a1674769ba75008f,"time, he will reconsider his attitude.",0,0,0,0,0,0,1
4118,70e664bbadf93c6c,No doubt you are another Wikipedia Democrat. H...,1,0,0,0,0,0,0
5344,89ab9a1eb6c8c026,"""""""While essentially every CNN program was dow...",0,0,0,0,0,0,1
2934,5302e5ea8863409c,WHAT THE FUCK IS YOUR PROBLEM WHY DO U KEEP DE...,1,0,1,0,1,0,0
4559,e6cba0f00fc7475f,"""\n\nNew Award\n\nWell Done!!!\n\n The IAmThe...",0,0,0,1,0,0,0
4143,8e660c1eed36b6f2,YO\nWHY CANT I MESS UP UR SITE\nNO SCHOOLS R A...,1,0,1,0,1,1,0
968,b5233a74b5954a3b,"Here we go again... Fuck you, !",1,0,1,0,1,0,0
2680,21e051df5488bb51,"""\n\n Of course... you called me a """"sock pupp...",1,0,0,0,0,0,0
5418,3d09050e81205163,Where was it before?,0,0,0,0,0,0,1
5180,9d1f8df102cdb499,"""\nI apologize for ShortJason's advertising. ...",0,0,0,0,0,0,1


In [6]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’\n])')
def clean_string(s): return re_tok.sub(' ', s)

In [7]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, data, label_cols):
        self.data = data
        self.label_cols = label_cols #test comment

    def __getitem__(self, item):

        comment = clean_string(self.data.comment_text[item])
        toxic = self.data.toxic[item]
        severe_toxic = self.data.severe_toxic[item]
        obscene = self.data.obscene[item]
        threat = self.data.threat[item]
        insult = self.data.insult[item]
        identity_hate = self.data.identity_hate[item]
        none = self.data.none[item]
        return comment, torch.FloatTensor([toxic, severe_toxic, obscene, threat, insult, identity_hate, none])
    def __len__(self):
        return len(self.data)

In [8]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# train['none'] = 1-train[label_cols].max(axis=1)
label_cols.append('none')
COMMENT = 'comment_text'
label_cols.append(COMMENT)
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [9]:
gpu = 0
device = torch.device(gpu if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)
print(device)

cuda:0


In [10]:
MAX_SEQ_LEN=512
BATCH_SIZE =4
LEARNING_RATE = 1e-1
WARMUP_STEPS = 4
EPOCHS = 1
model_name= 'bert-base-uncased'
DROPOUT_RATE = 0.3
# wandb.init(project=model_name)
# config = wandb.config

In [11]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_SEQ_LEN)
import transformers
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_SEQ_LEN)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [12]:
class AutoNet(nn.Module):
  def __init__(self, seqLength=MAX_SEQ_LEN, numClasses=7, model_name=model_name, len_t = len(tokenizer), DROPOUT_RATE = DROPOUT_RATE):
    super(AutoNet, self).__init__()
    self.model = AutoModel.from_pretrained(model_name)
    self.model.resize_token_embeddings(len_t)
    # for p in self.bert.parameters():
    #   p.requires_grad=False
    self.d1 = nn.Dropout(DROPOUT_RATE)
    self.l1 = nn.Linear(seqLength*768, 64)
    self.bn1 = nn.LayerNorm(64)
    self.d2 = nn.Dropout(DROPOUT_RATE)
    self.l2 = nn.Linear(64, numClasses)
    self.sigmoid = nn.Sigmoid()

  def forward(self,input_ids, attention_mask):
    x = self.model(input_ids, attention_mask=attention_mask)[0] #last_hidden_state #BSD
#     print("x1 view", x.view(x.shape[0],-1))
    x = self.d1(x.view(x.shape[0],-1))
#     print("x2 view", x)
#     print("x3 view", x.view(x.shape[0],-1))
    x = self.l1(x)
    x = self.bn1(x)
    x = nn.Tanh()(x)
    x = self.d2(x)
    x = self.l2(x)
    # x = self.drop(x)
    # x = F.softmax(x, dim=1)
    x = self.sigmoid(x)
    return x

In [13]:
train_set = MyData(train, label_cols)
train_set, val_set = torch.utils.data.random_split(train_set, [int(0.8*len(train_set)), len(train_set)-int(0.8*len(train_set))] )
train_set = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_set = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [14]:
model = AutoNet()
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)
loss_criteria = nn.BCELoss()
loss_criteria = loss_criteria.to(device)
# model.load_state_dict(torch.load(f'/kaggle/input/notebook294d5a41bc-f62f18-gpt/roberta-baseNetMLE_2.pt'))

In [15]:
from sklearn.metrics import accuracy_score


In [16]:
model.load_state_dict(torch.load(f'/kaggle/input/notebook985089b5b5/bert-base-uncasedNetMLE_2.pt'))
model.train()
from tqdm import tqdm
for epoch in tqdm(range(EPOCHS)):
  count = 0
  total_loss =0 
  model.train()
  for i,data in enumerate(train_set):
    optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    # loss, logits = out[:2]
    # print(out.shape, logits.shape, labels.shape)
    
    loss = loss_criteria(out, labels)
    loss.backward()
    optimizer.step()
    scheduler.step()
    total_loss += loss.detach().data
    if (i+1)%1024==0:
      print(f"Epoch: {epoch}, batch: {i+1}, loss: {total_loss/(1024*BATCH_SIZE)}")
#       wandb.log({"train_loss_per_batch": loss})
      total_loss = 0
  torch.save(model.state_dict(), f'{model_name}NetMLE_{epoch+2}.pt')

  model.eval()
  all_pred = []
  all_gold = []
  for i,data in enumerate(val_set):
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    all_pred.extend(1*(out>0.6).clone().detach().cpu().numpy())
    all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))

  count=0
  for i in range(len(all_gold)):
    if (all_gold[i]==all_pred[i]).all():
      count+=1
#   wandb.log({"val_acc": count/len(all_gold)})
  print("Validation accuracy:", count/len(all_gold))

  0%|          | 0/1 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13

Epoch: 0, batch: 1024, loss: 0.14427612721920013


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
100%|██████████| 1/1 [06:12<00:00, 372.05s/it]

Validation accuracy: 0.0


In [17]:
# model.load_state_dict(torch.load(f'/kaggle/input/notebook985089b5b5/roberta-baseNetMLE_1.pt'))
# model.eval()
# all_pred = []
# all_gold = []
# for i,data in enumerate(val_set):
# #     optimizer.zero_grad()
#     enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
#     input_ids = enc['input_ids'].to(device)
#     attention_mask = enc['attention_mask'].to(device)
#     labels = torch.tensor(data[1]).to(device)
#     out = model(input_ids=input_ids, attention_mask=attention_mask)

#     all_pred.extend(1*(out>0.8).clone().detach().cpu().numpy())
#     all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))


In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(all_gold, all_pred)

0.0

In [19]:
label_cols[:7]

['toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate',
 'none']

In [20]:
test_labels = test_labels[test_labels.toxic!=-1]
test_labels['none'] = 1-test_labels[label_cols[:6]].max(axis=1)
test_labels.sample(20)
# a = test[:30]
# print(a)
# a.to_csv(r'sample_debug.csv', index = False)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
42532,468b5f77924e612e,0,0,0,0,0,0,1
81603,883356bb132b52c8,0,0,0,0,0,0,1
112589,bbe21ac31e599123,0,0,0,0,0,0,1
58135,60b50a2b28be525c,0,0,0,0,0,0,1
25224,2a0995d88e94dd64,0,0,0,0,0,0,1
106336,b17ec3d4f1321397,0,0,0,0,0,0,1
63794,6a4e7f7647fd28f0,1,0,0,0,1,0,0
9657,1023e7329929cf6f,0,0,0,0,0,0,1
49627,526fe35c23ff5552,0,0,0,0,0,0,1
76058,7ecc2d5c21e4b6d4,0,0,0,0,0,0,1


In [21]:
# test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')

test_set = test.merge(test_labels, left_index=True, right_index=True)
# test_set = test_set[test_set.toxic==0][:200]
test_set = test_set[["id_x", "comment_text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate", "none"]]
test_set = test_set.reset_index(drop=True)
test_set = test_set.rename(columns={"id_x": "id"})

# test_set['none'] = 1-test_set[label_cols].max(axis=1)

In [22]:
test_set.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0,1
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0,1
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0,1
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0,1
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0,1


In [23]:
test_set = MyData(test_set, label_cols)
test_set = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [24]:
# model.load_state_dict(torch.load(f'/kaggle/input/bertnet/BertNetCE_2.pt'))
model.eval()
all_pred = []
all_gold = []
for i,data in enumerate(test_set):
#     optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    all_pred.extend(1*(out>0.9).clone().detach().cpu().numpy())
    all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))


/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [25]:
accuracy_score(all_gold, all_pred)
print(all_gold, all_pred)

[array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([1, 0, 1, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([1, 0, 1, 0, 1, 0, 0]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 1]), array([0,

In [26]:
print(sklearn.metrics.classification_report(all_gold, all_pred))
# count=0
# for i in range(len(all_gold)):
#     if (all_gold[i]==all_pred[i]).all():
#         count+=1
# count/len(all_gold)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      6090
           1       0.00      0.00      0.00       367
           2       0.00      0.00      0.00      3691
           3       0.00      0.00      0.00       211
           4       0.00      0.00      0.00      3427
           5       0.00      0.00      0.00       712
           6       0.00      0.00      0.00     57735

   micro avg       0.00      0.00      0.00     72233
   macro avg       0.00      0.00      0.00     72233
weighted avg       0.00      0.00      0.00     72233
 samples avg       0.00      0.00      0.00     72233



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# wandb.log({"test_acc": count/len(all_gold) })